# Tutorial 4

In [13]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
from scipy import special
import ipywidgets as widgets
from IPython.display import clear_output, display


Using matplotlib backend: Qt5Agg


In this tutorial, modulation formats, detection and the error probability will be investigated.



## Task 1

Generate equiprobable N BPSK symbols and add AWGN to simulate the channel.

In [14]:
# Simulation parameters
N = 100000  # number of symbols
SNR_dB = 1  # signal to noise ratio in dB
hist_bins = 100  # number of bins in the histogram


In [15]:
# generate N random symbols with values 0 and 1
bits = np.random.randint(2, size=N) # YOUR CODE HERE 
symbols = 2 * bits - 1  # YOUR CODE HERE # BPSK modulation

# generate noise with variance according to the required SNR and mean 0
sigma = np.sqrt(1 / (2 * 10 ** (SNR_dB / 10))) # YOUR CODE HERE # noise std. deviation
noise = sigma * np.random.randn(N) # YOUR CODE HERE # noise

# add noise to the symbols
received = symbols + noise # YOUR CODE HERE # AWGN channel


Plot the histogram of the received values and the conditional probabilities for the channel for the symbols $+1$ and $-1$ 

In [16]:
# plot the histogram of the received symbols for +1 and -1

# Normalize the histograms
hist_0 , bins_0 = np.histogram(received[bits == 0], bins=hist_bins, density=True)
hist_1, bins_1 = np.histogram(received[bits == 1], bins=hist_bins, density=True)

#plot the histograms
plt.figure()
plt.bar(bins_0[:-1], hist_0, width=0.05, label='Hist : -1', alpha=0.5)
plt.bar(bins_1[:-1], hist_1, width=0.05, label='Hist : +1', alpha=0.5)
plt.xlabel('Received Value')
plt.title('Histogram of received symbols')

# plot the gaussian pdf with the same variance as the noise with the mean +1 and -1

x = np.linspace(-4, 4, 1000)
gaussian_1 = 1 / (sigma * np.sqrt(2 * np.pi)) * np.exp(-(x - 1) ** 2 / (2 * sigma ** 2)) # YOUR CODE HERE # gaussian pdf with mean +1 
gaussian_m1 = 1 / (sigma * np.sqrt(2 * np.pi)) * np.exp(-(x + 1) ** 2 / (2 * sigma ** 2)) # YOUR CODE HERE # gaussian pdf with mean -1

plt.plot(x, gaussian_1, label='$\mathcal{N}(1,\sigma^2)$', color='r')
plt.plot(x, gaussian_m1, label='$\mathcal{N}(-1,\sigma^2)$', color='b')
plt.legend()
plt.show()


### Check

Check whether the histograms and pdfs match. 

## Task 2

In this task, you do the threshold detection on the received symbol and calculate the symbol error rate (SER).

In [17]:

detection_threshold = 0  # SET THE DETECTION THRESHOLD HERE


detected_bits = np.zeros(N)
detected_bits[received > detection_threshold] = 1 # YOUR CODE HERE # detected bits
    
    # calculate the SER
SER = np.sum(np.abs(bits - detected_bits)) / N

print('The simulated SER = ', SER)


The simulated SER =  0.05639


Calculate the analytical symbol error probability and compare it to the simulated SER.

In [18]:
# define the Q function in terms of the error function special.erf()
def Q(x):
    return 0.5 * (1 - special.erf(x / np.sqrt(2))) # YOUR CODE HERE # q function


SER_analytical = Q(np.sqrt(2 * 10 ** (SNR_dB / 10))) # YOUR CODE HERE # theoretical SER

print('The analytical error probability is: ', SER_analytical)


The analytical error probability is:  0.056281951976541456


### Questions

1)  What is the effect of the SNR on the received symbol pdfs? Change the SNR value and the comment.
2) What is the effect of the detection threshold on the SER? What is the optimal detection threshold?
3) Compare the simulated error rate and the theoretical error error probability.

### Answers

1) Conditional pdfs overlap more with the decreasing SNR.
2) For the uniform symbols, the optimal threshold is 0 due to nearest neighbor rule. Threshold is selected to minimize the error rate.
3) They are approximately equal. 

# Task 3

In this task, you do the detection for non-uniform symbols.

Generate $N$ non-uniform BPSK symbols with the probabilities $p_0$ and $p_1$ and simulate the AWGN channel.

In [19]:
p_0 = 0.9
p_1 = 0.1

bits = np.random.choice([0, 1], size=N, p=[p_0, p_1]) #YOUR CODE HERE # generate the bits according to the probabilities using np.random.choice

# generate the symbols according to the probabilities
symbols = 2 * bits - 1  #YOUR CODE HERE # BPSK modulation 

# generate noise with variance according to the required SNR and mean 0
noise = sigma * np.random.randn(N) #YOUR CODE HERE # noise
received = symbols + noise #YOUR CODE HERE # AWGN channel

hist_0 , bins_0 = np.histogram(received[bits == 0], bins=hist_bins, density=True)
hist_1, bins_1 = np.histogram(received[bits == 1], bins=hist_bins, density=True)

# plot the histogram of the received symbols for +1 and -1
plt.figure()
plt.bar(bins_0[:-1], hist_0 * p_0, width=0.05, label='Hist : -1', alpha=0.5)
plt.bar(bins_1[:-1], hist_1 * p_1, width=0.05, label='Hist : +1', alpha=0.5)
plt.xlabel('Received Value')
plt.title('Histogram of received symbols')


# plot the gaussian pdf with the same variance as the noise with the mean +1 and -1
gaussian_1 = 1 / (sigma * np.sqrt(2 * np.pi)) * np.exp(-(x - 1) ** 2 / (2 * sigma ** 2)) # YOUR CODE HERE # gaussian pdf with mean +1 
gaussian_m1 = 1 / (sigma * np.sqrt(2 * np.pi)) * np.exp(-(x + 1) ** 2 / (2 * sigma ** 2)) # YOUR CODE HERE # gaussian pdf with mean -1

gaussian_1 = gaussian_1 * p_1
gaussian_m1 = gaussian_m1 * p_0

plt.plot(x, gaussian_1, label='$p_1 \cdot \mathcal{N}(1,\sigma^2)$', color='r')
plt.plot(x, gaussian_m1, label='$p_0 \cdot \mathcal{N}(-1,\sigma^2)$', color='b')
plt.legend()
plt.show()




Do the threshold detection by setting the threshold to minimize error rate.

In [20]:
# Define the slider
threshold_slider = widgets.FloatSlider(min=-1, max=1, step=0.01, description='Threshold:', layout=widgets.Layout(width='50%'))

# Display the slider
display(threshold_slider)

# Create empty lists to store the threshold and SER values
threshold_values = []
ser_values = []

# Define a function to perform the detection and calculate the SER
def update_threshold(change):
    detection_threshold = change.new
    
    # Clear the output of the current cell
    clear_output(wait=True)

    detected_bits = np.zeros(N)
    detected_bits[received > detection_threshold] = 1
    SER_nonuniform = np.sum(np.abs(bits - detected_bits)) / N

    # Append the threshold and SER values to the lists
    threshold_values.append(detection_threshold)
    ser_values.append(SER_nonuniform)

    # Print the SER
    #print('The SER for non-uniform symbols for the threshold', detection_threshold, 'is:', SER_nonuniform)

    
    # Display the slider
    display(threshold_slider)

    #clear the previous plots
    plt.clf()

    # Plot on the first subplot
    plt.subplot(1, 2, 1)
    plt.plot(x, gaussian_1, label='$p_1 \cdot \mathcal{N}(1,\sigma^2)$', color='r')
    plt.plot(x, gaussian_m1, label='$p_0 \cdot \mathcal{N}(-1,\sigma^2)$', color='b')
    plt.axvline(x=detection_threshold, ymin=0, ymax=1, color='g', linestyle='--', label='Threshold')
    plt.ylim(0, 0.6)
    plt.xlim(-4, 4)
    plt.xlabel('Received Value')
    plt.ylabel('Conditional pdf')
    plt.legend()

    # Plot on the second subplot
    plt.subplot(1, 2, 2)
    plt.plot(threshold_values, ser_values, color='orange')
    plt.xlabel('Threshold')
    plt.grid()
    plt.ylabel('SER')

    # Display the plots
    plt.tight_layout()
    plt.show()    

# Call the function when the value of the slider changes
threshold_slider.observe(update_threshold, 'value')



FloatSlider(value=0.98, description='Threshold:', layout=Layout(width='50%'), max=1.0, min=-1.0, step=0.01)

### Questions
1) How do you decide on the threshold value and why?
2) Can you calculate the threshold value analytically? Derive and state the equation for the optimal threshold. 

### Answers
1) We decide on the threshold to minimize the error rate/probability. It is the point at which conditional probabilities intersect as MAP rule states.
3) With the MAP rule, the threshold can be calculated analytically. It is $$ x = 0.5 \sigma_n^2 \ln \left(\frac{p_0}{p_1}\right )$$

# Task 4

In this task, you will do the detection and the error rate calculation for M-QAM and M-PSK modulated symbols. Firstly generate the constellation maps for M-QAM and M-PSK modulations.

In [22]:
M = 16  # modulation order
SNR_dB = 17  # signal to noise ratio in dB

sigma = np.sqrt(1 / (2 * 10 ** (SNR_dB / 10))) # YOUR CODE HERE # noise std. deviation


# Generate the constellation diagram for M-QAM modulation
constellation_qam = (2 * np.arange(np.sqrt(M)) - (np.sqrt(M) - 1)) # YOUR CODE HERE # generate the constellation for one dimension (like ASK)
constellation_qam = constellation_qam + 1j * constellation_qam[:, None] # YOUR CODE HERE # generate the constellation by extending the one dimensional constellation to two dimensions
constellation_qam /= np.sqrt(np.mean(np.abs(constellation_qam) ** 2)) # YOUR CODE HERE # normalize the constellation
constellation_qam = constellation_qam.flatten()


# Generate the constellation diagram for M-PSK modulation
constellation_psk = np.exp(1j * 2 * np.pi * np.arange(M) / M) # YOUR CODE HERE # generate the constellation
constellation_psk = constellation_psk.flatten()


In [30]:
# plot the constellation diagrams
plt.figure()
plt.scatter(constellation_qam.real, constellation_qam.imag, label='QAM', marker='x')
plt.scatter(constellation_psk.real, constellation_psk.imag, label='PSK', marker='o')
plt.xlabel('In-Phase')
plt.ylabel('Quadrature')
plt.legend(loc=(1.1,0.87))
plt.grid()
plt.axis('square')
plt.show()


Generate uniform N M-QAM symbols and simulate the AWGN channel.  

In [31]:

symbols = np.random.randint(M, size=N) # YOUR CODE HERE  # generate N random symbols with values 0 and M-1 
symbols = constellation_qam[symbols] 

# generate noise with variance according to the required SNR and mean 0
noise = sigma * np.random.randn(2 * N)  # YOUR CODE HERE # generate noise with variance according to the required SNR and mean 0
noise = noise[::2] + 1j * noise[1::2] # YOUR CODE HERE # convert the noise to a complex number
received = symbols + noise # YOUR CODE HERE # add noise to the symbols

# plot the constellation diagram
plt.figure()
plt.scatter(np.real(received[0:2000]), np.imag(received[0:2000]), label='Rx')
plt.scatter(np.real(symbols[0:100]), np.imag(symbols[0:100]),color='r', label='Tx', marker='x')
plt.title('Constellation diagram')
plt.xlabel('In-phase')
plt.ylabel('Quadrature')
plt.grid()
plt.legend(loc=(1.1,0.87))
plt.axis('square')  
plt.show()


Firstly, do the detection by selecting the closest constellation point to the received signal. Calculate the SER for the received symbols.

In [25]:

detected_symbols = np.zeros(N, dtype=complex)

# do the detection by choosing the closest symbol in the constellation
for i in range(N):
    detected_symbols[i] = constellation_qam[np.argmin(np.abs(received[i] - constellation_qam))] 


#calculate the SER

SER = np.sum(np.abs(symbols - detected_symbols) > 0.01) / N

print('The SER for QAM is: ', SER)

#print (symbols[0:10])
#print (detected_symbols[0:10])

The SER for QAM is:  0.00252


Calculate the analytical symbol error probability using the equation from the lecture  $$ P_s \approx \frac{4(\sqrt{M}-1)}{\sqrt{M}} Q \left(\sqrt{\frac{3\log_2(M)}{M-1}\gamma_b} \right).$$

In [26]:
gamma_b = 10 ** (SNR_dB / 10) / np.log2(M) # YOUR CODE HERE # SNR per bit
SER_analytical = 4 * (np.sqrt(M)-1)/np.sqrt(M) * Q(np.sqrt(3 * np.log2(M) / (M-1) * gamma_b)) # YOUR CODE HERE

print('The SER for QAM is: ', SER)
print('The analytical error probability for QAM is: ', SER_analytical)

The SER for QAM is:  0.00252
The analytical error probability for QAM is:  0.0023180244461353805


Then, do the same for the M - PSK modulation by generating N symbols and simulating the channel.

In [27]:
symbols = np.random.randint(M, size=N)
symbols = constellation_psk[symbols]

# generate noise with variance according to the required SNR and mean 0
noise = sigma * np.random.randn( N) + 1j * sigma * np.random.randn(N) # YOUR CODE HERE # complex noise
received = symbols + noise # YOUR CODE HERE # simulate the AWGN channel

# plot the constellation diagram
plt.figure()
plt.scatter(np.real(received[0:2000]), np.imag(received[0:2000]), label='Rx')
plt.scatter(np.real(symbols[0:100]), np.imag(symbols[0:100]),color='r', label='Tx', marker='x')
plt.title('Constellation diagram')
plt.xlabel('In-phase')
plt.ylabel('Quadrature')
plt.grid()
plt.legend(loc=(1.1,0.87))
plt.axis('square')  
plt.show()


Calculate the symbol error rate after the detection.

In [28]:
detected_symbols = np.zeros(N, dtype=complex)

# do the detection by choosing the closest symbol in the constellation
for i in range(N):
    detected_symbols[i] = constellation_psk[np.argmin(np.abs(received[i] - constellation_psk) **2 )] # YOUR CODE HERE # do the detection by choosing the closest symbol in the constellation


#calculate the SER

SER = np.sum(np.abs(symbols - detected_symbols) > 0.01) / N

print('The SER for PSK is: ', SER)


The SER for PSK is:  0.05037


Calculate the analytical symbol error probability using the following equation from the lecture $$ P_s \approx 2 Q\left( \sqrt{\frac{2\log_2 (M) \pi^2}{M^2} \gamma_b }  \right) $$

In [29]:
SER_analytical = 2 * Q(np.sqrt(2 * gamma_b * (np.pi**2) * np.log2(M) / (M ** 2) ))  # YOUR CODE HERE


print('The SER for PSK is: ', SER)
print('The analytical error probability for PSK is: ', SER_analytical)


The SER for PSK is:  0.05037
The analytical error probability for PSK is:  0.04931881111324832


### Questions

1) Which modulation format do you choose by looking at their performance at the same SNR? Why?
2) Do the analytical and simulated error rates correspond to each other?  
3) Try the simulation also for SNR value of 5 dB and M = 4. Compare the simulated error rates obtained from the QAM and PSK modulation. Do they match? Why?
4) Now, for the parameter setting in the previous question, compare the analytical error probabilities with the simulated error rates for QAM and PSK. Comment on the result. 

### Answers

1) QAM since the error rate is small. We distribute the points more evenly in the same area, minimum distance between the points are larger.
2) Due to high SNR, they are the same.
3) They are the same modulation shifted by $\pi/4$, so the simulated error rates are the same.
4) Since the SNR value is small, the nearest neighbor assumption done in the PSK $P_s$ derivation does not hold. Thats why, we see lower value for the analytical SER compared to the simulated one. For QAM, we don't have nearest neighbor assumption, so the simulated and analytical $P_s$ values are approximately equal. 